### Solutions for Web Scraping Homework

In [1]:
# imports
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
# connect to homework
i = 1
url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C+United+Kingdom&ns=1&start={i}'
req = requests.get(url).text
scraper = BeautifulSoup(req)
restaurants = scraper.find_all('div', {'class': 'container__09f24__21w3G'})

### Scraping Titles, Categories

In [5]:
restaurant_links = [restaurant.find_all('a', {'class': 'link-color--inherit__09f24__3PYlA'}) for restaurant in restaurants]
titles = [link.text for link_group in restaurant_links for link in link_group if 'biz' in link['href']]

In [7]:
# no list comprehension
titles = []
for link_group in restaurant_links:
    for link in link_group:
        if 'biz' in link['href']:
            titles.append(link.text)

In [9]:
# restaurant categories
list_of_cats = []
for link_groups in restaurant_links:
    restaurant_cats = []
    for link in link_groups:
        if 'find_desc' in link['href']:
            restaurant_cats.append(link.text)
    list_of_cats.append(restaurant_cats)
    
category_one = []
category_two = []
category_three = []
for restaurant in list_of_cats:
    for i in range(3):
        if i == 0:
            try:
                category_one.append(restaurant[i])
            except:
                category_one.append(None)
        if i == 1:
            try:
                category_two.append(restaurant[i])
            except:
                category_two.append(None)
        if i == 2:
            try:
                category_three.append(restaurant[i])
            except:
                category_three.append(None)

### Scraping Phone Numbers, Neighborhoods, Reviews

In [12]:
# paragraphs
restaurant_paragraphs = [restaurant.find_all('p') for restaurant in restaurants]

In [13]:
# get the phone numbers
phone_numbers = [paragraph_group[0].text if paragraph_group[0].text.replace(' ', '').isdigit() else None for paragraph_group in restaurant_paragraphs]

In [67]:
# without a comprehension
phone_numbers = []
for paragraph_group in restaurant_paragraphs:
    if paragraph_group[0].text.replace(' ', '').isdigit():
        phone_numbers.append(paragraph_group[0].text)
    else:
        phone_numbers.append(None)

In [15]:
# finding out neighborhoods
neighborhoods = []
for paragraph_group in restaurant_paragraphs:
    if paragraph_group[-2].text != 'Delivery':
        neighborhoods.append(paragraph_group[-2].text)
    else:
        neighborhoods.append(paragraph_group[-3].text)

In [25]:
# reviews -- sometimes they are missing, hence the check
reviews = []
for idx, paragraph_group in enumerate(restaurant_paragraphs):
    if idx == len(restaurant_paragraphs) - 1:
        try:
            reviews.append(paragraph_group[1][3].text.replace('\xa0more', ''))
        except:
            reviews.append(paragraph_group[-1].text.replace('\xa0more', ''))
    else: 
        if '\xa0more' in paragraph_group[-1].text:
                reviews.append(paragraph_group[-1].text.replace('\xa0more', ''))
        else:
            reviews.append(None)

### Scraping Price Ranges, Addresses

In [27]:
restaurant_spans = [restaurant.find_all('span') for restaurant in restaurants]

In [28]:
# number of reviews
num_reviews = [int(span_group[4].text) if span_group[4].text.isdigit() else None for span_group in restaurant_spans]

In [29]:
# price ranges -- sometimes they have blanks:
price_ranges = []
for span_group in restaurant_spans:
    if '\xA3' in span_group[5].text:
        price_ranges.append(span_group[5].text)
    elif '\xA3' in span_group[6].text:
        price_ranges.append(span_group[6].text)
    else:
        price_ranges.append(None)

In [31]:
# for addresses

# helper function to check if a given piece of text is an address or not
def is_address(text):
    text = text.split()
    try:
        # accommodates for values like 34-66, 4/6, etc
        if text[0].replace('-', '').replace('/', '').isdigit() and text[-1].isalpha():
            return True
        else:
            return False
    except:
        return False

addresses = []
for span_group in restaurant_spans:
    has_address = False
    for group in span_group:
        if is_address(group.text):
            has_address = True
            address = group.text
    if has_address:
        addresses.append(address)
    else:
        addresses.append(None)

### Scraping Average Rating

In [33]:
# get all the divs
restaurant_divs = [restaurant.find_all('div', {'role': 'img'}) for restaurant in restaurants]

In [34]:
restaurant_ratings = [float(str(restaurant[0]).split('"')[1].split()[0]) for restaurant in restaurant_divs]

### Final DataFrame

In [42]:
# neighborhoods
neighborhoods = [paragraph_group[-2].text for paragraph_group in restaurant_paragraphs[:-1]]
neighborhoods.append(restaurant_paragraphs[-1][-2].text)

In [44]:
df_dict = {
    'Name': titles,
    'PhoneNumber': phone_numbers,
    'Address': addresses,
    'Neighborhood': neighborhoods,
    'PriceRange': price_ranges,
    'AvgRating': restaurant_ratings,
    'NumRatings': num_reviews,
    'CategoryOne': category_one,
    'CategoryTwo': category_two,
    'CategoryThree': category_three,
    'Review': reviews
}

df = pd.DataFrame(df_dict)
df

,Name,PhoneNumber,Address,Neighborhood,PriceRange,AvgRating,NumRatings,CategoryOne,CategoryTwo,CategoryThree,Review
0,Ffiona’s Restaurant,020 7937 4152,51 Kensington Church Street,Outdoor Seating,££,4.5,269,British,None,None,“Ffiona's is easily my favorite restaurant in ...
1,Dishoom,020 7420 9320,12 Upper Saint Martin's Lane,Covent Garden,££,4.5,1842,Indian,None,None,“Hard to find a way to add any higher praise t...
2,Restaurant Gordon Ramsay,020 7352 4441,68 Royal Hospital Road,Chelsea,££££,4.5,205,French,British,None,“Compared to Michelin 3-star restaurants in Ca...
3,Flat Iron,None,17 Beak Street,Soho,££,4.5,380,Steakhouses,None,None,“Ok guys...here is how it is. We were in Londo...
4,The Golden Chippy,020 8692 4333,62 Greenwich High Road,Deptford,££,5.0,108,Fish & Chips,None,None,“Loved the service !!!! The Cod was extremely ...
5,The Queens Arms,020 7834 3313,11 Warwick Way,Victoria,££,4.5,118,British,Pubs,Gastropubs,“The staff at The Queens Arms were amazing ton...
6,Duck & Waffle,020 3640 7310,110 Bishopsgate,Aldgate,£££,4.0,704,Modern European,Bars,British,“Probably the best restaurant we visited in Lo...
7,Padella,None,6 Southwark Street,London Bridge,££,4.5,205,Italian,None,None,“A great dining experience even in COVID times...
8,The Grazing Goat,020 7724 7243,6 New Quebec Street,Marylebone,££,4.0,242,Gastropubs,British,None,“My wife and I have visited London a few times...
9,Mother Mash,020 7494 9644,26 Ganton Street,Soho,££,4.0,468,British,None,None,“Soho is full of culture and amazing places to...


### Final Answer, That Loops Through The Entire Website

In [45]:
i = 0
num_round = 1
total_titles = []
total_phone_numbers = []
total_addresses = []
total_neighborhoods = []
total_price_ranges = []
total_restaurant_ratings = []
total_num_reviews = []
total_category_one = []
total_category_two = []
total_category_three = []
total_reviews = []
looping = True

# helper function to check if a given piece of text is an address or not
def is_address(text):
    text = text.split()
    try:
        # accommodates for values like 34-66, 4/6, etc
        if text[0].replace('-', '').replace('/', '').isdigit() and text[-1].isalpha():
            return True
        else:
            return False
    except:
        return False

while looping:
    url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C+United+Kingdom&ns=1&start={i}'
    req = requests.get(url)
    scraper = BeautifulSoup(req.text)
    restaurants = scraper.find_all('div', {'class': 'container__09f24__21w3G'})
    if len(restaurants) == 0:
        print("No more restaurants found.  Breaking the loop.")
        looping = False
        break
    
    #### Getting All Links, Code for Titles and Categories
    restaurant_links = [restaurant.find_all('a', {'class': 'link-color--inherit__09f24__3PYlA'}) for restaurant in restaurants]
    titles = [link.text for link_group in restaurant_links for link in link_group if 'biz' in link['href']]
    total_titles.extend(titles)
    
    # restaurant categories
    list_of_cats = []
    for link_groups in restaurant_links:
        restaurant_cats = []
        for link in link_groups:
            if 'find_desc' in link['href']:
                restaurant_cats.append(link.text)
        list_of_cats.append(restaurant_cats)
    category_one = []
    category_two = []
    category_three = []
    for cat in list_of_cats:
        for idx in range(3):
            if idx == 0:
                try:
                    category_one.append(cat[idx])
                except:
                    category_one.append(None)
            if idx == 1:
                try:
                    category_two.append(cat[idx])
                except:
                    category_two.append(None)
            if idx == 2:
                try:
                    category_three.append(cat[idx])
                except:
                    category_three.append(None)
    total_category_one.extend(category_one)
    total_category_two.extend(category_two)
    total_category_three.extend(category_three)
                    
    # getting all the paragraphs within each restaurant
    restaurant_paragraphs = [restaurant.find_all('p') for restaurant in restaurants]
    
    # get the phone numbers
    phone_numbers = [paragraph_group[0].text if paragraph_group[0].text.replace(' ', '').isdigit() else None for paragraph_group in restaurant_paragraphs]
    total_phone_numbers.extend(phone_numbers)
    
    # get the neighborhoods
    neighborhoods = []
    for paragraph_group in restaurant_paragraphs:
        if '\xa0more' in paragraph_group[-1].text:
            if paragraph_group[-2].text != 'Delivery':
                neighborhoods.append(paragraph_group[-2].text)
            else:
                neighborhoods.append(paragraph_group[-3].text)
        else:
            if paragraph_group[-1].text != 'Delivery':
                neighborhoods.append(paragraph_group[-1].text)
            else:
                neighborhoods.append(paragraph_group[-2].text)
            
    total_neighborhoods.extend(neighborhoods)
    
    # get the reviews
    reviews = []
    for paragraph_group in restaurant_paragraphs:
        if '\xa0more' in paragraph_group[-1].text:
            reviews.append(paragraph_group[-1].text.replace('\xa0more', ''))
        else:
            reviews.append(None)
                
    total_reviews.extend(reviews)
    
    # get all the spans in a given page
    restaurant_spans = [restaurant.find_all('span') for restaurant in restaurants]
    
    # get the number of reviews
    num_reviews = [int(span_group[4].text) if span_group[4].text.isdigit() else None for span_group in restaurant_spans]
    total_num_reviews.extend(num_reviews)
    
    # get the price ranges
    price_ranges = []
    for span_group in restaurant_spans:
        if '\xA3' in span_group[5].text:
            price_ranges.append(span_group[5].text)
        elif '\xA3' in span_group[6].text:
            price_ranges.append(span_group[6].text)
        else:
            price_ranges.append(None)
    total_price_ranges.extend(price_ranges)
            
    # get the addresses
    addresses = []
    for span_group in restaurant_spans:
        has_address = False
        for group in span_group:
            if is_address(group.text):
                has_address = True
                address = group.text
        if has_address:
            addresses.append(address)
        else:
            addresses.append(None)
    total_addresses.extend(addresses)
            
    # get all the rating divs
    restaurant_divs = [restaurant.find_all('div', {'role': 'img'}) for restaurant in restaurants]
    
    # and get the restaurant ratings
    restaurant_ratings = [float(str(restaurant[0]).split('"')[1].split()[0]) for restaurant in restaurant_divs]
    total_restaurant_ratings.extend(restaurant_ratings)
    
    print(f'Finished Round {num_round}, value of i: {i}')
    i += 10
    num_round += 1
    
df_dict = {
    'Name': total_titles,
    'PhoneNumber': total_phone_numbers,
    'Address': total_addresses,
    'Neighborhood': total_neighborhoods,
    'PriceRange': total_price_ranges,
    'AvgRating': total_restaurant_ratings,
    'NumRatings': total_num_reviews,
    'CategoryOne': total_category_one,
    'CategoryTwo': total_category_two,
    'CategoryThree': total_category_three,
    'Review': total_reviews
}

df = pd.DataFrame(df_dict)
df

Finished Round 1, value of i: 0
Finished Round 2, value of i: 10
Finished Round 3, value of i: 20
Finished Round 4, value of i: 30
Finished Round 5, value of i: 40
Finished Round 6, value of i: 50
Finished Round 7, value of i: 60
Finished Round 8, value of i: 70
Finished Round 9, value of i: 80
Finished Round 10, value of i: 90
Finished Round 11, value of i: 100
Finished Round 12, value of i: 110
Finished Round 13, value of i: 120
Finished Round 14, value of i: 130
Finished Round 15, value of i: 140
Finished Round 16, value of i: 150
Finished Round 17, value of i: 160
Finished Round 18, value of i: 170
Finished Round 19, value of i: 180
Finished Round 20, value of i: 190
Finished Round 21, value of i: 200
Finished Round 22, value of i: 210
Finished Round 23, value of i: 220
Finished Round 24, value of i: 230
No more restaurants found.  Breaking the loop.


,Name,PhoneNumber,Address,Neighborhood,PriceRange,AvgRating,NumRatings,CategoryOne,CategoryTwo,CategoryThree,Review
0,The Mayfair Chippy,020 7741 2233,14 North Audley Street,Mayfair,££,4.5,280.0,Fish & Chips,None,None,“One of the best fish and chips I've ever had....
1,Ffiona’s Restaurant,020 7937 4152,51 Kensington Church Street,Outdoor Seating,££,4.5,269.0,British,None,None,“Ffiona's is easily my favorite restaurant in ...
2,Dishoom,020 7420 9320,12 Upper Saint Martin's Lane,Covent Garden,££,4.5,1842.0,Indian,None,None,“Hard to find a way to add any higher praise t...
3,Restaurant Gordon Ramsay,020 7352 4441,68 Royal Hospital Road,Chelsea,££££,4.5,205.0,French,British,None,“Compared to Michelin 3-star restaurants in Ca...
4,Flat Iron,None,17 Beak Street,Soho,££,4.5,380.0,Steakhouses,None,None,“Ok guys...here is how it is. We were in Londo...
...,...,...,...,...,...,...,...,...,...,...,...
235,Canton Arms,020 7582 8710,177 South Lambeth Road,Oval,££,4.0,29.0,Gastropubs,None,None,“This place is in the running for 'best pub ev...
236,The Kings Arms,020 7629 0416,2 Shepherd Market,Mayfair,££,4.0,48.0,Pubs,British,None,“Perfect little pub. Not on the main streets ...
237,Flat Iron,020 6666 66666,47 - 51 Caledonian Road,King's Cross,None,4.5,3.0,Steakhouses,None,None,“We loved Flat Iron!! Super helpful about my g...
238,The Gate,020 7833 0401,51 Queen Caroline Street,Hammersmith,£££,4.5,66.0,Vegetarian,None,None,“Brilliant menu - I'm vegan but my boyfriend i...
